In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Final Project: Part 1
Contributions From: Ryan Cottone

Welcome to the final project of the semester! In Part 1, we will use what we learned about cryptography to recreate the Transport Layer Security (TLS) protocol that is used to encrypt your connections to websites. We will also build a server authentication and file storage system.

In Part 2, you will be presented with 8 different challenges of varying difficulty. Each challenge will involve finding and exploiting some flaw in the setup of the aforementioned system.

Please see the spec [here](https://codebreakingatcal.org/docs/ProjectTest/Project%20Spec/Part%201/) for a guideline on completing this part.

### Helpers

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
import random
import math
import json
import string
import numpy as np
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

### Utility Functions

In [ ]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modularInverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def findSquareRoot(N,p):
    N = N % p
    
    if pow(N,(p-1)//2,p)==1:
        if (p%4) == 3:
            x1= pow(N,(p+1)//4, p)
            y1= p - x1
            return (x1,y1)
        else:
            for x in range(1,((p-1)//2)+1):
                if N == pow(x, 2, p):
                    return (x,p-x)
    return []



def extendedGCD(a,b):
    s_n1 = 1 # s_i-1
    s = 0 # s_i
    
    t_n1 = 0 #t_i-1
    t = 1 #t_1
    
    
    while b > 0:
        q = a // b

        s, s_n1 = s_n1 - q*s, s
        t, t_n1 = t_n1 - q*t, t
        
        a,b = b, a % b
    
    return (a, s_n1, t_n1)

egcd = extendedGCD

def millerRabinWitnessQ(a,n):
    q = n-1
    k = 0
    
    while q % 2 == 0:
        k+=1
        q //= 2
    
    i = 0
        
    b_i = pow(a, q, n)
    
    if b_i == 1 or b_i == (n-1):
        return False
    
    for i in range(k):
        if b_i == 1:
            return False
        
        b_i = pow(b_i, 2, n)

    return True

def probablyPrime(p):
    if p < 3:
        return True if (p == 1 or p == 2) else False
    
    for i in range(20):
        a = random.randrange(2, p) # randrange() returns [2, p) = [2, p-1]
        
        if millerRabinWitnessQ(a, p):
            return False
    return True

def findPrime(lBound, uBound): 
    p = random.randrange(lBound, uBound+1) 
    
    while not probablyPrime(p):
        p = random.randrange(lBound, uBound+1)
    
    return p

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def getExpansion(n,m):
    arr = []
    
    while n > 0:
        r = n % m
        n //= m
        
        arr.append(r)
    
    return arr

def textToInt(s):
    total = 0
    
    for i in range(len(s)):
        total += ord(s[i])*(256**i)
    
    return total

def intToText(n):
    expansion = getExpansion(n, 256)
    
    finalStr = ""
    
    for i in range(len(expansion)):
        finalStr += chr(expansion[i])
        
    return finalStr


def bxor(b1, b2): # use xor for bytes
    result = bytearray()
    for b1, b2 in zip(b1, b2):
        result.append(b1 ^ b2)
    return result

def getRandomBytes(n):
    return get_random_bytes(n)

### Elliptic Curve Functions

In [ ]:
def evaluateP(x, E, p):
    evaled = ((pow(x, 3, p) + E[0]*x + E[1])%p)
    
    if evaled == 0:
        return [0]
    
    return findSquareRoot(evaled, p)

def pointOnCurve(P,E,p):
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    
    if P[0] == 'O':
        return True
    
    evaled = evaluateP(P[0], E, p)
    
    if len(evaled) == 0:
        return False
    
    return (evaled[0]%p == P[1]) or (evaled[1]%p == P[1])

def isElliptic(E, p):
    """
    Takes in curve E as [A, B] and modulo p. 
    Returns whether the discriminant of E is nonzero.
    """
    
    # Make sure to do this modulo p.
    discriminant = (4 * pow(E[0], 3, p) + 27 * pow(E[1], 2, p)) % p 
    
    return discriminant != 0 

def addPoints(P,Q,E,p):
    """
    Given points P,Q of the form (x,y), curve E of the form [A,B], and prime p, find P + Q.
    """
    
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    assert pointOnCurve(P, E, p), "Point P is not on the elliptic curve"
    assert pointOnCurve(Q, E, p), "Point Q is not on the elliptic curve"

    multi = 1 # Lambda

    if P == 'O': # If P is the point at infinity
        return Q 
    elif Q == 'O': # If Q is the point at infinity
        return P 
    elif P[0] == Q[0] and P[1] == (-Q[1] % p): # If P and Q are on a vertical line (share the same x-coord)
        return 'O' 
    elif P[0] == Q[0]: # If P = Q
        multi = ((3*pow(P[0], 2, p) + E[0])*modularInverse(2*P[1], p)) % p # Set lambda
    else: 
        multi = ((Q[1] - P[1])*modularInverse((Q[0] - P[0])%p, p))%p # Set lambda

    x_3 = (pow(multi, 2, p) - P[0] - Q[0]) % p 
    y_3 = (multi*(P[0] - x_3) - P[1]) % p 

    return (x_3, y_3)

def messageToPoint(m, E, p, K=100):
    """
    Koblitz's algorithm for finding a point for a given number (plaintext, m). Uses tolerance K=100 by default.
    
    Returns a point if found, otherwise None.
    """
    
    for j in range(K):
        square_roots = evaluateP(m*K + j, E, p)
        
        if square_roots == 0:
            return (m*K + j, 0)
        elif len(square_roots) == 2:
            return (m*K + j, square_roots[0])
        
    return None # Failure case

def pointToMessage(P, K=100):
    """
    Returns the message corresponding to point P.
    """
    
    return P[0]//K

def doubleAndAdd(P,n,E,p):
    assert isElliptic(E, p), "Invalid elliptic curve (has zero discriminant)"
    assert pointOnCurve(P, E, p), "Point P is not on the elliptic curve"
    
    point = P
    finalPoint = 'O'
        
    # What this while loop does is iteratively find the binary representation
    # starting from the least significant bit. It is faster than finding the representation first.
    while n > 0:
        r = n % 2
        n //= 2
        
        if r == 1: # Checks if this binary bit is 1
            # If the bit is one, add the current power of two point to the finalPoint
            finalPoint = addPoints(finalPoint, point, E, p) 
        
        # Double the current power of two point to the next power of two
        point = addPoints(point, point, E, p) 
    
    return finalPoint

def generateEllipticKeypair(P, q, E, p):
    assert pointOnCurve(P, E, p), "Generator point P is not on the given curve"
    
    secret = random.randrange(2,q)
    
    V = doubleAndAdd(P, secret, E, p)
    
    return secret, V

def generateECDH(generatorPoint, E, p, N):
    """
    Given a generator point P, curve E, prime p, and bound N, returns nP for some random integer n in [0, N-1].
    Returns (nP, n)
    """
    
    n = random.randrange(2, N) # [2, N-1]
    
    nP = doubleAndAdd(generatorPoint, n, E, p)
    
    return nP, n

def combineECDH(publicPoint, k, E, p):
    """
    Takes in point publicPoint, secret k, the curve E, and prime p. 
    publicPoint can be either Bob's public or Alice's public key (aP, bP),
    and k is the opposite person's private key (n).
    Returns the shared secret (ab)P.
    """
    
    return doubleAndAdd(publicPoint, k, E, p)


### Hashing/MAC Functions

In [ ]:
def hash_SHA256(M): # M is integer
    h = SHA256.new()
    h.update(bytes(int_to_bytes(M)))
    return int(h.hexdigest(), 16)

def generateHMAC(key, data):
    if type(key) == int:
        key = key.to_bytes(32, byteorder='big')

    h = HMAC.new(key, digestmod=SHA256)

    h.update(data)

    return h.hexdigest()

def verifyHMAC(key, data, hmac):
    if type(key) == int:
        key = key.to_bytes(32, byteorder='big')

    h = HMAC.new(key, digestmod=SHA256)

    h.update(data)

    try:
        h.hexverify(hmac)
        return True
    except:
        return False

### Symmetric Encryption Functions

In [ ]:
def encryptAES(key, data):
    if type(key) == int:
        key = key.to_bytes(32, byteorder='big')
    
    assert len(key) == 16 or len(key) == 24 or len(key) == 32, "Invalid keysize"
    
    iv = get_random_bytes(16)
    
    paddedData = pad(data, AES.block_size)
    
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    
    encrypted = cipher.encrypt(paddedData)

    iv = b64encode(cipher.iv).decode('utf-8')

    ct = b64encode(encrypted).decode('utf-8')

    return json.dumps({'iv':iv, 'ciphertext':ct})

def decryptAES(key, encrypted):
    if type(key) == int:
        key = key.to_bytes(32, byteorder='big')
    
    assert len(key) == 16 or len(key) == 24 or len(key) == 32, "Invalid keysize"
    
    try:
        b64 = json.loads(encrypted)
        iv = b64decode(b64['iv'])
        ct = b64decode(b64['ciphertext'])
        
        cipher = AES.new(key, AES.MODE_CBC, iv)

        return unpad(cipher.decrypt(ct), AES.block_size)
        
    except (ValueError, KeyError):
        raise AssertionError("Invalid ciphertext")

### Asymmetric Encryption Functions

In [ ]:
def generateRSAKeypair(b):
    p = findPrime(2, 2**(b//2) -1)
    q = findPrime(2, 2**(b//2) -1)
    
    while p == q:
        q = findPrime(2, 2**(b//2) -1) # ensure P!=q
    
    N = p*q
    
    e = 65537
    
    while math.gcd(e, (p-1)*(q-1)) != 1:
        e += 2 # Since e must be odd for gcd(e, p-1 * q-1) = 1, we start at 65537 and inc by 2 each time
    d = modularInverse(e, (p-1)*(q-1))
    
    return (e, N, d)

# Doesnt use padding
def encryptRSA(message, e, N):
    return pow(message, e, N)

def decryptRSA(encrypted, d, N):
    return pow(encrypted, d, N)

# Finds H(M)^d mod N.
def signRSA(d, M, N):
    assert type(d) == int, "Private key must be an integer"
    assert type(M) == int, "Message must be an integer"
    assert type(N) == int, "Modulus must be an integer"
    
    return pow(hash_SHA256(M%N), d, N)

# Finds S^e mod N and compares it to H(M).
def verifyRSA(e, S, N, M):
    assert type(e) == int, "Public exponent must be an integer"
    assert type(S) == int, "Signature must be an integer"
    assert type(M) == int, "Message must be an integer"
    assert type(N) == int, "Modulus must be an integer"
    
    return pow(S, e, N) == hash_SHA256(M%N)
    

In [ ]:
CERT_AUTH = { 'e': 65537, 'N': 33790420484761320225234266446986435791020053290995177788399417698648848366075439013295931744537889745793682732187585867814285806211190774412138926826806937374931229955338241741978503726324443629746710612128866806815968501932728765477787763877641403710570749219182260822344263730489611164845428107854720086677, 'd': 13990616901200824332998639242549657982350872729162978917073076266121984534132734754806980909837734993242162151729712109543321259006983256231951260806039426156135728347335452488348561573956342300497866864117403272359970143321087093698235850894812096199039329431651823513521315154736696406982035194667449777653}

# Transport Layer Security

First: what even is TLS? TLS is a protocol designed to securely encrypt communications between a client and a web server. It uses a combination of public-key encryption, symmetric encryption, digital signatures, and more!

We will be using the **RSA-ECDH** version of TLS. In order to do this, the client and server accomplish the following steps:

Before the connection begins, the server needs to configure certain constants and get a certificate signed by a certificate authority.
1. The server generates an RSA keypair of the form ((e, N), d).
2. The certificate authority issues a signature saying the server's RSA public key corresponds to that server.
3. The server picks an elliptic curve (we will use the Sepc256k1 curve) and a generator point.

After the server is setup, the client and server initiate a connection and trade RSA keys.
1. The client asks the server for its RSA public key and certificate.
2. The server provides this, and the client verifies the certificate is valid for that RSA key.
3. The server then sends the elliptic curve parameters, signed by their RSA private key.
4. The client verifies these parameters are correctly signed by the server's RSA public key.

Now that the client and server have securely traded RSA keys, they are able to sign messages to ensure integrity and authenticity. We can begin the Elliptic-Curve Diffie Hellman Key Exchange (ECDH).
1. The server generates a secret value a and public point aP using the aforementioned generator point, and sends AP to the client. They attach a signature of aP using their RSA private key.
2. The client receives nP, and computes their secret b. They compute bP and send it to the server (alongside a signature on bP).
3. The server computes a(bP) = (ab)P and the client computes b(aP) = (ab)P. They now convert this point to an integer and use it as a shared secret.

Once we have securely established a shared secret, we can use symmetric encryption to send requests. 
1. The client create requests and sends it encrypted via AES using the shared secret derived from ECDH. The server can decrypt this, and encrypt its response using the same method.

In [ ]:
class Server:
    RSA_PUBLIC_KEY = (
        65537,
        9038494040587010144527283006157928881319808198037800262507236135386268739708006022679916006469762927636026303078260115925689450875819797050237830498287577
                     )
    
    RSA_PRIVATE_KEY = 2535417770757764232372393774283341693305817384267313426399332119458339022396333163725804069143378584578179197504985944189947045947523212247400435285708353
    
    SIGNED_SERVER_CERTIFICATE = 12700340844416036644598467941678481034795616611285372846483555754936821137468034013552250333992213649482476234812345812443409067513447778032927158222423442969388095714455462826384483204764801308373583214095146933570191105218603640338334545613351980554082339092228528550094782514990171966181657825617151691397
    
    ELLIPTIC_CURVE = [0,7]
    
    ELLIPTIC_CURVE_MODULUS = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    
    GENERATOR_POINT = (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
    
    GENERATOR_POINT_ORDER = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
    
    LOGINS = {}

    def signMessage(self, message):
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
    
    def publishRSAPublicKey(self):
        """
        Returns ((e, N), signature) where e,N are the server's public key, and signature is the cert authorities' signature 
        on e||N.
        """
        
        return ((self.RSA_PUBLIC_KEY[0], self.RSA_PUBLIC_KEY[1]), self.SIGNED_SERVER_CERTIFICATE)
    
    # ------------
    # QUESTION 1.2
    # ------------
    def generateECDHParameters(self):
        """
        Returns a generator point P, curve E, and modulus N signed via the server's RSA keys.
        """
        # DON'T FILL THIS IN
        pass
    
    # ------------
    # QUESTION 1.4
    # ------------
    def generateSignedECDHMessage(self):
        # DON'T FILL THIS IN
        pass
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        return verifyRSA(publicKey[0], signature, publicKey[1], expected)
    
    def acceptConnection(self, clientPublicKey):
        self.CLIENT_PUBLIC_KEY = clientPublicKey
    
    # ------------
    # QUESTION 1.5
    # ------------
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        # DON'T FILL THIS IN 
        
    
    # ------------
    # QUESTION 1.8
    # ------------
    def createLogin(self, username, password):
        """
        Given a username and password, creates a login entry if one does not already exist.
        """
        # DON'T FILL THIS IN
        pass
        
    # ------------
    # QUESTION 1.9
    # ------------
    def verifyLogin(self, username, password):
        # DON'T FILL THIS IN
        pass
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    # ------------
    # QUESTION 1.7
    # ------------
    def verifyRequestIntegrity(self, requestedData):
        """
        Takes in a request of the form { data: ..., hmac: ...} and returns True if the HMAC is valid.
        """
        # DON'T FILL THIS IN 
        pass
    
    def handleRequest(self, requestData):
        if 'data' not in requestData or 'hmac' not in requestData:
            raise AssertionError("Invalid requested format")
            
        assert self.verifyRequestIntegrity(requestData), "Request MAC check failed"
        
        # Request verified, time to decrypt
        request = json.loads(self.decryptMessage(requestData['data']))
        
        if request['type'] == 'createLogin':
            assert 'username' in request and 'password' in request, "Bad request data"
            
            return self.createLogin(request['username'], request['password'])
        elif request['type'] == 'verifyLogin':
            return self.verifyLogin(request['username'], request['password'])
        else:
            raise AssertionError("Invalid request type")

In [ ]:
class Client:
    def __init__(self, keybits=512):
        rsa_keypair = generateRSAKeypair(keybits)
        
        self.RSA_PUBLIC_KEY = (rsa_keypair[0], rsa_keypair[1])
        self.RSA_PRIVATE_KEY = (rsa_keypair[2])
        
    def signMessage(self, message):
        return signRSA(self.RSA_PRIVATE_KEY, message, self.RSA_PUBLIC_KEY[1])
        
    # ------------
    # QUESTION 1.1
    # ------------
    def verifyServerCertificate(self, serverPublicKey, signature):
        """
        Asserts the server's certificate is correctly signed by the given certificate authority.

        The certificate is on int(e||N).
        """
        # DON'T FILL THIS IN
        pass
    # ------------
    # QUESTION 1.3
    # ------------
    def verifyECDHParameters(self, ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature):
        """
        Verifies that the given ECDH parameters have a valid signature, and updates the state if so.
        """
        
        # DON'T FILL THIS IN
        pass
    
    # ------------
    # QUESTION 1.4
    # ------------
    def generateSignedECDHMessage(self):
        """
        Assumes we have already setup a valid connection.
        """
        # DON'T FILL THIS IN 
        pass
    
    def verifyECDHMessage(self, P, signature, publicKey):
        """
        Verifies point P was signed by the given publicKey (e, N).
        """

        expected = int(str(P[0]) + str(P[1]))

        return verifyRSA(publicKey[0], signature, publicKey[1], expected)
    
    # ------------
    # QUESTION 1.5
    # ------------
    def acceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        # DON'T FILL THIS IN 
        pass
    
    def encryptMessage(self, message):
        """
        Given string message, encrypts with the current setup
        """
        
        return encryptAES(self.SHARED_SECRET_INT, bytes(message, 'utf-8'))
    
    def decryptMessage(self, encrypted):
        """
        Given encrypted message, decrypts with the current setup
        """
        
        return decryptAES(self.SHARED_SECRET_INT, encrypted)
    
    # ------------
    # QUESTION 1.6
    # ------------
    def generateRequest(self, obj):
        """
        Given an object, encrypts and HMAC's the object in preparation for sending to the server.
        """
        # DON'T FILL THIS IN
        pass

**Question 1.1:** Implement Client.verifyServerCertificate.

In [ ]:
def verifyServerCertificate(self, serverPublicKey, signature):
    """
    Asserts the server's certificate is correctly signed by the given certificate authority.

    The certificate is on int(e||N).
    """
        
    e, N = CERT_AUTH['e'], CERT_AUTH['N']
        
    expected = int(str(serverPublicKey[0]) + str(serverPublicKey[1]))
        
    verified = ...

    if not verified:
            return False
        
    self.SERVER_PUBLIC_KEY = serverPublicKey
        
    return True

In [ ]:
# DON'T CHANGE THIS
Client.verifyServerCertificate = verifyServerCertificate

In [ ]:
grader.check("q1_1")

**Question 1.2:** Implement Server.generateECDHParameters in the Server class above. Run the below test when you are finished.

In [ ]:
def generateECDHParameters(self):
        """
        Returns a generator point P, curve E, and modulus N signed via the server's RSA keys.
        """

        intToSign = int(str(self.ELLIPTIC_CURVE[0]) + str(self.ELLIPTIC_CURVE[1]) 
                        + str(self.ELLIPTIC_CURVE_MODULUS) + str(self.GENERATOR_POINT[0]) + str(self.GENERATOR_POINT[1]))

        signature = ...

        return self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT, self.GENERATOR_POINT_ORDER, signature

In [ ]:
# DON'T CHANGE THIS
Server.generateECDHParameters = generateECDHParameters

In [ ]:
grader.check("q1_2")

**Question 1.3:** Implement Client.verifyECDHParameters

In [ ]:
def verifyECDHParameters(self, ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature):
        expected = int(str(ellipticCurve[0]) 
                        + str(ellipticCurve[1])
                        + str(ellipticCurveModulus) 
                        + str(generatorPoint[0]) 
                        + str(generatorPoint[1]))
        verified = ...
        
        if not verified:
            return False
        
        self.ELLIPTIC_CURVE = ...
        self.ELLIPTIC_CURVE_MODULUS = ...
        self.GENERATOR_POINT = ...
        self.GENERATOR_POINT_ORDER = ...
        
        return True

In [ ]:
# DON'T CHANGE THIS
Client.verifyECDHParameters = verifyECDHParameters

In [ ]:
grader.check("q1_3")

**Question 1.4:** Implement generateSignedECDHMessage in both Server and Client classes.

In [ ]:
def generateSignedECDHMessage(self):
        """
        Assumes we have already setup a valid connection.
        """
        ECDH = ...
        
        nP, n = ECDH[0], ECDH[1]
        
        intToSign = int(str(nP[0]) + str(nP[1]))
        
        self.ECDH_SECRET = ...
        
        return nP, self.signMessage(intToSign)

In [ ]:
# DON'T CHANGE THIS
Server.generateSignedECDHMessage = generateSignedECDHMessage
Client.generateSignedECDHMessage = generateSignedECDHMessage

In [ ]:
grader.check("q1_4")

**Question 1.5:** Implement acceptECDHMessage in both Server and Client.

Consider using self.verifyECDHMessage.

Note: The code you fill in should be the exact same for both functions.

In [ ]:
def clientAcceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        ...
        
        # Signature verified
        sharedSecretPoint = ...
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True
    
def serverAcceptECDHMessage(self, P, signature):
        """
        Given the sender's public point, verify its signature is as expected and then update internal state
        """
        
        ...
        
        # Signature verified
        sharedSecretPoint = ...
        
        self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
        
        return True

In [ ]:
# DON'T CHANGE THIS
Server.acceptECDHMessage = serverAcceptECDHMessage
Client.acceptECDHMessage = clientAcceptECDHMessage

In [ ]:
grader.check("q1_5")

**Question 1.6:** Implement Client.generateRequest

In [ ]:
def generateRequest(self, obj):
        """
        Given an object, encrypts and HMAC's the object in preparation for sending to the server.
        """
    
        encStr = self.encryptMessage(json.dumps(obj))
        
        encStrBytes = bytes(encStr, 'utf-8')
        
        hmac = ...
        
        requestObj = {'data': encStr, 'hmac': hmac}
        
        return requestObj

In [ ]:
# DON'T CHANGE THIS
Client.generateRequest = generateRequest

In [ ]:
grader.check("q1_6")

**Question 1.7:** Implement Server.verifyRequestIntegrity

In [ ]:
def verifyRequestIntegrity(self, requestedData):
        """
        Takes in a request of the form { data: ..., hmac: ...} and returns True if the HMAC is valid.
        """
        
        requestedDataBytes = bytes(requestedData['data'], 'utf-8')
        existingHMAC = requestedData['hmac']
        
        verified = ...
        
        return verified

In [ ]:
# DON'T CHANGE THIS
Client.verifyRequestIntegrity = verifyRequestIntegrity
Server.verifyRequestIntegrity = verifyRequestIntegrity

In [ ]:
grader.check("q1_7")

**Question 1.8:** Implement Server.createLogin

In [ ]:
def createLogin(self, username, password):
    if username in self.LOGINS: 
        return False
        
    salt = ''.join(np.random.choice(list(string.ascii_lowercase), 5))
        
    intToHash = textToInt(password + salt)
        
    hashedPassword = ...
        
    self.LOGINS[username] = (hashedPassword, salt)
        
    return True

In [ ]:
# DON'T CHANGE THIS
Server.createLogin = createLogin

In [ ]:
grader.check("q1_8")

**Question 1.9:** Implement Server.verifyLogin

In [ ]:
def verifyLogin(self, username, password):
    """
    Returns True if username,password is a valid entry. Otherwise False (errors if username does not exist).
    """
        
    assert username in self.LOGINS, "Username does not exist"
        
    hashedPassword = ...
    salt = ...
        
    intToHash = textToInt(password + salt)
        
    isCorrect = ...
                
    return isCorrect

In [ ]:
# DON'T CHANGE THIS
Server.verifyLogin = verifyLogin

In [ ]:
grader.check("q1_9")

Congrats on finishing Part 1 of this project! Here is an example integration test:

In [ ]:
# New client, server
client = Client()
server = Server()

# Accept the connection from the client
server.acceptConnection(client.RSA_PUBLIC_KEY)

# Server publishes its public key
server_PK, signature = server.publishRSAPublicKey()
assert client.verifyServerCertificate(server_PK, signature)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()
assert client.verifyECDHParameters(ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature)

# Once ECDH parameters are verified, generate our secret + public value
client_ECDH = client.generateSignedECDHMessage()
server_ECDH = server.generateSignedECDHMessage()

# client gets server ecdh message
client.acceptECDHMessage(server_ECDH[0], server_ECDH[1])

# server gets client ecdh message
server.acceptECDHMessage(client_ECDH[0], client_ECDH[1])
assert client.SHARED_SECRET_INT == server.SHARED_SECRET_INT

signupRequest = client.generateRequest({'type': 'createLogin', 'username': 'asciabsjf', 'password': 'aidojsd1r'})
server.handleRequest(signupRequest)

loginRequest = client.generateRequest({'type': 'verifyLogin', 'username': 'asciabsjf', 'password': 'aidojsd1r'})
assert server.handleRequest(loginRequest)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)